In [74]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.simplefilter(action="ignore")
import time
import pandas as pd

In [105]:
driver = webdriver.Chrome()
url = "https://sg.sbiz.or.kr/godo/index.sg"
driver.get(url)

In [84]:
df = pd.read_csv('data/스타벅스_매장정보.csv', encoding='utf-8')
address_list = (df['address']).tolist()

In [109]:
def input_address(driver, address):
    
    #검색 창에 주소 입력
    search_box = driver.find_element(By.CSS_SELECTOR, "#searchAddress")
    search_box.clear()
    search_box.send_keys(address)

    #검색 버튼 클릭
    time.sleep(1)
    element = driver.find_element(By.ID, "layerPopAddressMove")
    driver.execute_script("arguments[0].click();", element)

    #첫 번째 검색 결과 클릭
    time.sleep(1)
    address_list_element = driver.find_element(By.CSS_SELECTOR, "#adrsList > li > label")
    driver.execute_script("arguments[0].click();", address_list_element)

    #다음 버튼
    time.sleep(1)
    next_button = driver.find_element(By.CSS_SELECTOR, "#container > div:nth-child(1) > div:nth-child(3) > div.foot > a:nth-child(2) > span")
    driver.execute_script("arguments[0].click();", next_button)

def zoom_in_out(driver):
    
    #가끔 검색이 제대로 되지 않음
    zoom_in = driver.find_element(By.CSS_SELECTOR, "#container > div.custom_zoomcontrol > div:nth-child(3) > a:nth-child(2)")
    zoom_out = driver.find_element(By.CSS_SELECTOR, "#container > div.custom_zoomcontrol > div:nth-child(3) > a:nth-child(1)")
    driver.execute_script("arguments[0].click();", zoom_in)
    driver.execute_script("arguments[0].click();", zoom_out)

def select_map_options(driver):
    
    #맵 포인터 클릭
    time.sleep(1)
    sanggwon_button = driver.find_element(By.CSS_SELECTOR, "#map > div:nth-child(1) > div > div:nth-child(6) > div:nth-child(2) > div > ul > li.child > label")
    sanggwon_button.click()

    #반경 내 상권 정보 열람
    time.sleep(1)
    bankyung_button = driver.find_element(By.CSS_SELECTOR, "#map > div:nth-child(1) > div > div:nth-child(6) > div:nth-child(2) > div > ul > li.child > div > ul > li:nth-child(2) > label")
    bankyung_button.click()

    #반경 거리 700m
    time.sleep(1)
    bankyung = driver.find_element(By.CSS_SELECTOR, "#auto_circle > div > div.midd > ul > li:nth-child(7) > label")
    bankyung.click()

    #확인 버튼
    time.sleep(1)
    element = driver.find_element(By.CSS_SELECTOR, "#auto_circle > div > div.foot > a:nth-child(2) > span")
    element.click()

    #분석 버튼
    time.sleep(1)
    analyze_button = driver.find_element(By.CSS_SELECTOR, "#map > div:nth-child(1) > div > div:nth-child(6) > div:nth-child(3) > img")
    analyze_button.click()

def analyze_sales_data(driver):
    sales_data = []

    # 첫 번째 태그로 이동하여 데이터 수집
    some_tag1 = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > span:nth-child(3)")
    action = ActionChains(driver)
    action.move_to_element(some_tag1).perform()

    time.sleep(1)
    upso = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > div.analysis-section.analysis-01 > div.analysis-content > div:nth-child(1) > div > div > span").text
    sales_data.append(upso)
    SalesAmt = driver.find_element(By.CSS_SELECTOR, "#salesSmryInfoCurSaleAmt").text
    sales_data.append(SalesAmt)

    # 두 번째 태그로 이동하여 데이터 수집
    time.sleep(1)
    some_tag2 = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > span:nth-child(5)")
    action.move_to_element(some_tag2).perform()

    time.sleep(1)
    flowpop = driver.find_element(By.CSS_SELECTOR, "#flowPopSmryInfoFlowPop").text
    sales_data.append(flowpop)
    emppop = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoEmpPop").text
    sales_data.append(emppop)
    sales_info = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoAbodePop").text
    sales_data.append(sales_info)
    sales_info = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoEmpAvgCo").text
    sales_data.append(sales_info)

    return sales_data

def close_report(driver):
    time.sleep(1)
    driver.find_element(By.XPATH ,'//*[@id="bind"]/div[1]/a').click()

In [106]:
# 실행
address = '서울특별시 노원구 동일로 1015 (공릉동)'
sales_data = []

input_address(driver, address)
zoom_in_out(driver)
select_map_options(driver)

In [107]:
element = driver.find_element(By.CSS_SELECTOR, "#auto_circle > div > div.foot > a:nth-child(2) > span")
element.click()

In [108]:
input_search_value(driver, '1000')

In [ ]:
sales_data = analyze_sales_data(driver)
close_report(driver)

# 주소에 대한 정보를 sales_data에 추가
sales_data.insert(0, address)

print('sales_data:', sales_data)
